In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics


%matplotlib inline

### Reading in, filtering, and examining the heart attack cost disparities data

In [ ]:
# your path to the data file may vary!

ha_costs_df = pd.read_csv('../data/mmd_heart_attack_data.csv') 
tn_ha_costs = ha_costs_df.loc[ha_costs_df.state == 'TENNESSEE']
print(tn_ha_costs.shape)
print(tn_ha_costs.head(2))

### Now getting the cancer data

In [ ]:
cancer_costs_df = pd.read_csv('../data/mmd_cancer_data.csv')
tn_cancer_costs = cancer_costs_df.loc[cancer_costs_df.state == 'TENNESSEE']
print(tn_cancer_costs.shape)
print(tn_cancer_costs.head(2))

### Getting the income data and cleaning it a bit

In [ ]:
income_df = pd.read_csv('../data/irs_county_2016.csv')
tn_income = income_df.loc[income_df.STATE == 'TN']
tn_income.head(2)

In [ ]:
tn_income = tn_income[['STATE', 'COUNTYNAME', 'agi_stub', 'N1', 'mars1', 'MARS2', 'MARS4', 'N2', 'NUMDEP', 'ELDERLY', 'A00100', 'N02650', 'A02650', 'N02300', 'A02300']]
tn_income.columns = ['state', 'county', 'income_bucket', 'return_count', 'single_returns', 'joint_returns', 'head_of_house_returns', 'exemptions', 'dependents', 'elderly', 'agi', 'returns_with_total_inc','total_inc_amt', 'returns_with_unemployment', 'unemployment_comp']


### Week two coding tasks
#### Replacing coded values in the `income_bucket` column with descriptive text
- create a dictionary mapping codes to descriptions
- use `replace()` to update the df with text

In [ ]:
income_dict = {0:'Total', 1: 'Under $1', 2: 'Between 1 and $10,000', 3: 'Between 10,000 and $25,000',
              4: 'Between 25,000 and $50,000', 5: 'Between 50,000 and $75,000', 
               6: 'Between 75,000 and $100,000', 7: 'Between 100,000 and $200,000', 
               8:'$200,000 or more'}

In [ ]:
tn_income.income_bucket = tn_income.income_bucket.replace(income_dict)
tn_income.head(2)

#### Creating a new df that aggregates by county to get the totals for each county

In [ ]:
income_county_agg = tn_income.groupby('county').agg('sum').reset_index()
income_county_agg.head(2)

In [ ]:
income_county_agg['avg_income'] = round(income_county_agg.total_inc_amt * 1000 / income_county_agg.returns_with_total_inc, 0)
income_county_agg.head(3)

### Create a merged DataFrame for Heart Attack Costs and Income, keeping just `county`, `urban`, `analysis_value`, and `avg_income`; then do the same for Cancer Costs

In [ ]:
# we only need the county and the average income from income_county_agg
county_incomes = income_county_agg[['county', 'avg_income']]
county_incomes.head(2)

In [ ]:
# we only need county, urban, and analysis_value columns from the heart attack costs
tn_ha_costs = tn_ha_costs[['county', 'urban', 'analysis_value']]

In [ ]:
tn_ha_costs2 = pd.merge(tn_ha_costs, county_incomes, on= 'county', how = 'left')
tn_ha_costs2.head(2)

In [ ]:
tn_ha_costs2['cost_income_ratio'] = tn_ha_costs2.analysis_value / tn_ha_costs2.avg_income
tn_ha_costs2.describe()

In [ ]:
# we only need county, urban, and analysis_value columns from the cancer costs
tn_cancer_costs = tn_cancer_costs[['county', 'urban', 'analysis_value']]

In [ ]:
tn_cancer_costs2 = pd.merge(tn_cancer_costs, county_incomes, on= 'county', how = 'left')
tn_cancer_costs2.head(2)

In [ ]:
tn_cancer_costs2['cost_income_ratio'] = tn_cancer_costs2.analysis_value / tn_cancer_costs2.avg_income
tn_cancer_costs2.describe()

### Week 5 Coding Tasks

#### logistic regression model for myocardial infarction costs

- create target column (1 for cost-income ratio above the mean 0 if at or below the mean)
- encode the urban column
- split train/test
- use urban column to predict

In [ ]:
# create target variable
ha_cost_income_ratio_mean = tn_ha_costs2.cost_income_ratio.mean()
tn_ha_costs2['cost_ratio_above_mean'] = [1 if ratio > ha_cost_income_ratio_mean else 0 for ratio in tn_ha_costs2.cost_income_ratio]

In [ ]:
tn_ha_costs2.cost_ratio_above_mean.value_counts(normalize = True)

In [ ]:
tn_ha_costs2.head(2)

In [ ]:
# encode urban/rural
tn_ha_costs2 = pd.get_dummies(tn_ha_costs2, columns = ['urban'], drop_first = True)
tn_ha_costs2.head(2)

In [ ]:
X = tn_ha_costs2[['urban_Urban']]
y = tn_ha_costs2.cost_ratio_above_mean

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 99)

In [ ]:
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

In [ ]:
y_pred = logistic_model.predict(X_test)

In [ ]:
print(metrics.accuracy_score(y_test, y_pred))

#### the naive model (predicting the cost-income ratio above the mean for all cases) would have accuracy .527473

In [ ]:
print('                 Pred Below Mean:  Pred Above Mean:')
print('    Actual Below Mean:    ', metrics.confusion_matrix(y_test, y_pred)[0])
print('    Actual Above Mean:   ', metrics.confusion_matrix(y_test, y_pred)[1])

In [ ]:
y_pred_prob = logistic_model.predict_proba(X_test)[:,1]
print('Area Under Curve:', metrics.roc_auc_score(y_test, y_pred_prob))

#### Let's add another predictor - the Health Factors z-score from the county health rankings: [Robert Wood Johnson Foundation](https://www.countyhealthrankings.org)

![health factors](../data/health_factors.png)

In [ ]:
health_rankings = pd.read_excel('../data/2018 County Health Rankings Tennessee Data - v3.xlsx', 
                                sheet_name = 'Outcomes & Factors Rankings',
                               header = [0,1])

In [ ]:
health_rankings.head(2)

In [ ]:
health_rankings.columns = ['fips', 'state', 'county', 'outcomes_z_score', 'outcomes_rank', 'factors_z_score', 'factors_rank']
health_rankings.head()

In [ ]:
health_factors = health_rankings[['county', 'factors_z_score']]

#### let's see if `health_factors` and `tn_ha_costs2` can be merged as is

In [ ]:
tn_ha_costs2.head(2)

In [ ]:
tn_ha_costs2.county = tn_ha_costs2.county.str[0:-7]

In [ ]:
tn_ha_costs2.head(2)

In [ ]:
ha_with_health_factors =pd.merge(tn_ha_costs2, health_factors, on = 'county', how = 'left')
ha_with_health_factors.head(2)

In [ ]:
X = ha_with_health_factors[['urban_Urban', 'factors_z_score']]
y = tn_ha_costs2.cost_ratio_above_mean

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 99)
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

In [ ]:
y_pred = logistic_model.predict(X_test)

In [ ]:
print(metrics.accuracy_score(y_test, y_pred))

In [ ]:
print('                 Pred Below Mean:  Pred Above Mean:')
print('    Actual Below Mean:    ', metrics.confusion_matrix(y_test, y_pred)[0])
print('    Actual Above Mean:   ', metrics.confusion_matrix(y_test, y_pred)[1])